# Imports et Références

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from matplotlib.ticker import FuncFormatter

data = pd.read_excel('publications_scientifiques_par_pays.xlsx')

# 1. Le nuage de points entre le nombre moyen de documents produits par an et le nombre moyen de citations par an, pour chaque pays

In [ ]:
mean_docs_citations = data.groupby('Country').agg({
    'Documents': 'mean',
    'Citations': 'mean',
    'Rank': 'median',
    'H.index': 'mean'
}).reset_index().rename(columns={
    'Documents': 'Mean_Documents',
    'Citations': 'Mean_Citations',
    'Rank': 'Median_Rank',
    'H.index': 'Mean_H_index'
})

sizes = np.clip(mean_docs_citations['Mean_H_index'] * 10, 10, 200)

norm = plt.Normalize(mean_docs_citations['Median_Rank'].min(), mean_docs_citations['Median_Rank'].max())
colors = plt.cm.plasma_r(norm(mean_docs_citations['Median_Rank']))

extreme_docs = pd.concat([
    mean_docs_citations.nlargest(5, 'Mean_Documents'),
    mean_docs_citations.nsmallest(5, 'Mean_Documents')
])

extreme_cites = pd.concat([
    mean_docs_citations.nlargest(5, 'Mean_Citations'),
    mean_docs_citations.nsmallest(5, 'Mean_Citations')
])

extreme_values = pd.concat([extreme_docs, extreme_cites]).drop_duplicates()

def format_fn(tick_val, tick_pos):
    if int(tick_val) in [1, 10, 100, 1000, 10000, 100000, 1000000, 10000000]:
        return int(tick_val)
    else:
        return ''

plt.figure(figsize=(20, 15))
scatter = plt.scatter(mean_docs_citations['Mean_Documents'], mean_docs_citations['Mean_Citations'],
                      c=colors, s=sizes, alpha=0.6, edgecolor='black', linewidth=0.5)

plt.xscale('log')
plt.yscale('log')

plt.gca().xaxis.set_major_formatter(FuncFormatter(format_fn))
plt.gca().yaxis.set_major_formatter(FuncFormatter(format_fn))

for i, row in extreme_values.iterrows():
    plt.annotate(row['Country'], 
                 (row['Mean_Documents'], row['Mean_Citations']),
                 fontsize=9, ha='right', va='center')

plt.axhline(mean_docs_citations['Mean_Citations'].mean(), color='red', linestyle='--', label='Moyenne des citations')
plt.axvline(mean_docs_citations['Mean_Documents'].mean(), color='blue', linestyle='--', label='Moyenne des documents')

plt.title('Nuage de points par Pays : Documents Moyens vs Citations Moyennes')
plt.xlabel('Nombre moyen de documents produits par an (log scale)')
plt.ylabel('Nombre moyen de citations par an (log scale)')
plt.colorbar(scatter, label='Rang Médian des Pays')
plt.legend()

plt.show()

# 2. Une représentation graphique de l’évolution des rangs des pays, uniquement sur les 10 premiers rangs, avec le pays en 1er en haut

In [ ]:
top_ranks = data[data['Rank'] <= 10]

# les années en index et les pays en colonnes avec leurs rangs
pivot_data = top_ranks.pivot(index='Year', columns='Country', values='Rank')

# le premier rang soit en haut
pivot_data_sorted = pivot_data.sort_values(by=pivot_data.index[0], axis=1, ascending=True)

colormap = plt.cm.get_cmap('tab20', len(pivot_data_sorted.columns))
country_colors = {country: colormap(i) for i, country in enumerate(pivot_data_sorted.columns)}

plt.figure(figsize=(20, 10))

labeled_countries = set()

countries = pivot_data_sorted.columns.tolist()

for country in countries:
    line_data = pivot_data_sorted[country].dropna()
    plt.plot(line_data.index, line_data, color=country_colors[country], marker='o')

    # si la ligne est isolée, ajouter l'étiquette lors de la première apparition
    if len(line_data) <= 3 and line_data.index[0] > pivot_data_sorted.index[0]:
        if country not in labeled_countries:
            plt.text(line_data.index[0], line_data.iloc[0], ' ' + country,
                     verticalalignment='center', horizontalalignment='left',
                     color=country_colors[country], fontsize=12)
            labeled_countries.add(country)

for country in countries:
    line_data = pivot_data_sorted[country].dropna()
    if not line_data.empty:
        end_year = line_data.index[-1]
        last_rank = line_data.iloc[-1]
        if country not in labeled_countries:
            offset = 0.3 if last_rank <= 5 else -0.3
            plt.text(end_year + 0.2, last_rank + offset, country,
                     verticalalignment='center', color=country_colors[country], fontsize=12)
            labeled_countries.add(country)
            
plt.gca().invert_yaxis()

plt.xticks(pivot_data_sorted.index)
plt.yticks(np.arange(1, 11, 1))

plt.xlabel('Année', fontsize=14)
plt.ylabel('Rang', fontsize=14)
plt.title('Evolution du classement du top 10 des pays au fil des années', fontsize=16)

plt.legend().remove()
plt.tight_layout()
plt.show()

# 2 BIS. Une représentation graphique de l’évolution des rangs des pays, uniquement sur les 10 premiers rangs, avec le pays en 1er en haut

In [ ]:
top_countries = data[data['Rank'] <= 10]

plt.figure(figsize=(15, 8))
sns.lineplot(data=top_countries, x='Year', y='Rank', hue='Country', palette='Set1', marker='o')

plt.xlabel('Année')
plt.ylabel('Rang')
plt.title('Evolution du classement du top 10 des pays au fil des années')
plt.legend(title='Country', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.gca().invert_yaxis()
plt.xticks(top_countries['Year'].unique())
plt.yticks(range(1, top_countries['Rank'].max() + 1))

for country, rank in top_countries.groupby('Country'):
    plt.text(rank['Year'].iloc[-1] + 0.5, rank['Rank'].iloc[-1] + 0.25, country, fontsize=10, ha='right', va='center')

plt.grid(True)
plt.show()


# 3. Une carte des pays dont le H-index moyen est supérieur à un seuil choisi par l’utilisateur

In [ ]:
def update_map(hindex):
    colorscale = px.colors.sequential.Plasma
    filtered_data = data[data['H.index'] >= hindex]
    fig = px.choropleth(filtered_data, 
                        locations='Country', 
                        locationmode='country names', 
                        color='H.index', 
                        color_continuous_scale=colorscale, 
                        range_color=(0, data['H.index'].max() + 100), 
                        title=f'Carte du monde dégradé par H.Index (H-index > {hindex})',
                        width=1400,
                        height=1000, 
                        hover_data={'H.index': True, 'Country': True}, 
                       )
    
    fig.show()
    
index_slider = widgets.IntSlider(value=data['H.index'].min(), min=data['H.index'].min(), max=data['H.index'].max(), step=1, description='Index supérieur à ')

widgets.interactive(update_map, hindex=index_slider)

# 4. Une carte des pays représentant le TOP réalisé plus haut avec le choix de l'année variable

In [ ]:
def update_map(year):
    filtered_data = data[(data['Year'] == year) & (data['Rank'] <= 10)]
    
    fig = px.choropleth(filtered_data, 
                        locations='Country', 
                        locationmode='country names', 
                        color='Documents',  
                        color_continuous_scale='rdbu', 
                        range_color=(filtered_data['Documents'].min(), filtered_data['Documents'].max()),  
                        title=f'Top 10 des pays par production de document sur l\'année : {year}',
                        width=1400,  
                        height=1000,  
                        hover_data={'Documents': True, 'Country': True}  
                       )
    
    fig.show()

year_slider = widgets.IntSlider(value=data['Year'].min(), min=data['Year'].min(), max=data['Year'].max(), step=1, description='Year:')

widgets.interactive(update_map, year=year_slider)
